# Table of Contents

1. [Header](#header)
2. [Particles](#particles)
3. [FFT Data](#fft)
4. [Halo and Subhalo](#halo)<br>
    4.1. [FOF Data](#fof)<br>
    4.2. [SUBFIND Catalog](#subcat)

### This notebook gives examples of how to use read_indra.py to read in Indra data on the SciServer.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import indratools as indra

Initialize the X, Y, and Z that specify the Indra run, the snapshot of the particle data (`snapnum`) and FFT data (`tnum`) for redshift 0, and the data directory (since we are not reading from the datascope). Note that when `datadir` is specified, it determines which simulation is read, and X, Y, and Z are ignored.

In [ ]:
X = 2 ; Y = 0; Z = 0; snapnum = 63; tnum = 504
datadir = '/home/idies/workspace/indra/2_0_0/'

# Header
<a id="header"></a>

Length data are in units of Mpc/h, velocities in km/s, and masses in MSun/h.

In [ ]:
header = indra.getheader(X,Y,Z,snapnum,datadir)
print(header.keys())

In [ ]:
print(header['num_files'],header['npart'],header['mass'],header['time'],header['BoxSize'])

# Particles
<a id="particles"></a>

Read positions only, no sorting:

In [ ]:
pos_ns = indra.getpos(X,Y,Z,snapnum,datadir)
print(pos_ns[0])

Get velocities too, and sort particles and velocities by their IDs:

In [ ]:
pos,vel,ids = indra.getparticles(X,Y,Z,snapnum,datadir,sort=True)

In [ ]:
print(pos[0],vel[0],np.max(ids))

# FFT Data
<a id="fft"></a>

Before reading, define some functions to calculate the power spectrum of this gridded FFT data.

In [ ]:
def powspec(fft_re,fft_im,nbins=100,k=k):
    # PS = fft_re^2+fft_im^2
    # Requires k's already defined from kx,ky,kz = indra.getkvals()
    ps = fft_re*fft_re+fft_im*fft_im
    ps = ps[k>0] # ignore k = 0
    k = k[k>0]
    
    # average PS in logarithmic bins of k
    ps1d, kbin = np.histogram(np.log10(k),nbins,weights=ps)
    counts = np.histogram(np.log10(k),nbins)[0]
    ps1d = ps1d[counts>0]/counts[counts>0]
    
    binvals = kbin[0:nbins]+np.diff(kbin)//2
    binvals = binvals[counts>0]

    return binvals,ps1d

In [ ]:
# the cosmology is hard-coded here but om = header['omega0']
def growthfunc(a, om=0.272):
    ol = 1-om
#    a = 1./(1.+z)
    da=a/10000.
    integral = 0.
    for i in range(10000):
        aa=(i+1)*da
        integral+=da/((aa*np.sqrt(om/(aa**3)+ol))**3)

    return 5*om/2*np.sqrt(om/a**3+ol)*integral

Read the CAMB linear power spectrum, which is normalized to z=0. (So to compare to Plin, multiply PS(z) by D(z)^2/D(z=0)^2.)

In [ ]:
kth,pth = np.loadtxt('pk_indra7313_CAMB.txt',unpack=True)

Compute the k vectors. These arrays have the same shape as the FFT data.

In [ ]:
kx,ky,kz = indra.getkvals()
k = np.sqrt(kx*kx+ky*ky+kz*kz)

Now plot the ratio of power spectra from the FFT data to the linear theory PS at different redshifts.

In [ ]:
fft_re,fft_im,a = indra.getfft(X,Y,Z,tnum,datadir)
bins,ps = powspec(fft_re,fft_im) # need bins to interpolate linear PS
plin = np.interp(10**bins,kth,pth)
norm = header['BoxSize']**3/header['npart']**2
plt.figure(figsize=(8,6))
for i in np.arange(0,501,100):
    fft_re,fft_im,a = indra.getfft(X,Y,Z,i,datadir)
    bins,ps = powspec(fft_re,fft_im)
    normL = growthfunc(a)**2/growthfunc(1)**2
    plt.plot(10**bins,(ps*norm)/(plin*normL),label="{:.3f}".format(a)) 
plt.xscale('log')
plt.legend(loc='upper left')
plt.xlabel(r'$k$ (h/Mpc)',size='large')
plt.ylabel(r'$P(a,k)/P_{lin}(k)$',size='large');

# Halo and Subhalo
<a id="halo"></a>

The FOF (or 'group') and SUBFIND headers contain the total number of groups/subhalos, and the number of files (NTask, which is always 256 for Indra).

In [ ]:
TotNgroups, NTask = indra.getfofheader(X,Y,Z,snapnum,datadir)
print(TotNgroups,NTask)
TotNsubs,NTask = indra.getsubheader(X,Y,Z,snapnum,datadir)
print(TotNsubs,NTask)

## FOF Data
<a id="fof"></a>

If all you want is the number of particles in each FOF halo (to multiply by `header['mass']`, for example), this is given by `groupLen`. `groupOffset` then gives the information needed to index `groupids`, the particle IDs in each group.

In [ ]:
groupLen,groupOffset = indra.getfof(X,Y,Z,snapnum,datadir)
print(np.min(groupLen),np.max(groupLen))

In [ ]:
groupLen,groupOffset,groupids = indra.getfofids(X,Y,Z,snapnum,datadir)

The IDs of the particles in halo i are given by:

In [ ]:
i = 0
haloIDs = groupids[groupOffset[i]:groupOffset[i]+groupLen[i]]

Let's plot some particles! I know that halo 0 doesn't cross any boundaries, so we can ignore periodic boundary conditions for now. This will only work if you called `getparticles` with `sort=True`.

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(pos[haloIDs,0],pos[haloIDs,1],'m.')
plt.xlabel('x (Mpc/h)')
plt.ylabel('y (Mpc/h)');

## Subhalo Catalog
<a id="subcat"></a>

There isn't much to the FOF data, since the unbinding procedure and calculation of halo properties is done by SUBFIND.

In [ ]:
cat = indra.getsubcat(X,Y,Z,snapnum,datadir)
subids = indra.getsubids(X,Y,Z,snapnum,datadir)

In [ ]:
print(cat.keys())

In [ ]:
print(np.mean(cat['M200crit']),np.mean(cat['R200crit']),np.mean(np.abs(cat['SubVel'])))

Indexing can get a bit tricky, so here are some examples. First let's pick a FOF group with 5 subhalos.

In [ ]:
hasSubs = np.where(cat['NsubPerHalo'] == 5)[0]
thishalo = np.max(hasSubs)
print(thishalo,cat['M200crit'][thishalo],cat['R200crit'][thishalo])

The IDs of the particles in this FOF halo (before unbinding) can be found via:

In [ ]:
haloIDs = groupids[groupOffset[thishalo]:groupOffset[thishalo]+groupLen[thishalo]]

Let's get the indexes of the subhalos with this parent halo. We can do this in two ways.

In [ ]:
print(np.where(cat['subParentHalo'] == thishalo)[0])
subindices = cat['FirstSubOfHalo'][thishalo]+np.arange(cat['NsubPerHalo'][thishalo])
print(subindices)

The halo positions are calculated for every subhalo, so let's define the center as that of the main subhalo, and take care of periodic boundary conditions. (Eventualy I will write some PBC utility functions...)

In [ ]:
boxsize = header['BoxSize']
center = cat['SubPos'][cat['FirstSubOfHalo'][thishalo],:]
print(center)
halopos = pos[haloIDs,:] - center
halopos[halopos < boxsize/2.] += boxsize
halopos[halopos > boxsize/2.] -= boxsize
print(np.mean(halopos,axis=0))

Now let's plot the particles in these halos! We'll plot the main halo in black first, so any particles that remain black were unbound by SUBFIND and don't belong to any subhalos (including the main `FirstSubOfHalo`).

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(halopos[:,0],halopos[:,1],marker = '.',color='k')
subindices = cat['FirstSubOfHalo'][thishalo]+np.arange(cat['NsubPerHalo'][thishalo])
colors = ['g','b','m','r','c']
for c, i in zip(colors,subindices):
    subIDs = subids[cat['subOffset'][i]:cat['subOffset'][i]+cat['subLen'][i]]
    subpos = pos[subIDs,:] - center
    subpos[subpos < boxsize/2.] += boxsize
    subpos[subpos > boxsize/2.] -= boxsize
    plt.scatter(subpos[:,0],subpos[:,1],marker ='.',color=c)
plt.xlabel('x (Mpc/h)',size='large')
plt.ylabel('y (Mpc/h)',size='large');